# Les bases de données 


## Introduction 

Une base de données est **un conteneur dans lequel il est possible de stocker des données de façon structurée.** Cette structure permet au programme informatique connectée à celle-ci de faire des recherches complexes.


## Et python dans tout ca ! 


### SQLite
Notre exemple de cours concernera le module python SQLite et plus particulièrement SQLite3.
SQLite a été conçu pour être intégré dans le programme même. Pour des projets plus ambitieux type projets web ou infrastructure le choix de MySQL & SQL serait plus judicieux. 


In [1]:
import sqlite3
import os 

#### Créer une base de données avec SQLite :

In [2]:
conn = sqlite3.connect('./db/ma_base_exemple.db')

Lorsque vous executerez votre programme vous remarquerez que si la base n'existe pas encore, un fichier sera crée dans le dossier de votre programme. Et si celui-ci existe déjà il sera réutilisé. Vous pouvez bien évidemment choisir l'emplacement de votre base de données en renseignant un path, exemple: "/data/ma_base_exemple.db" . *Il vous faudra cependant vérifier que le dossier existe avant de l'utiliser.*
Il est également possible de travailler avec une base de données de manière temporaire:

In [3]:
#Lorsque le travail que vous attendiez est terminé, pensez à fermer la connexion vers la base
#conn.close()

#### Un exemple de création de table : 

In [4]:
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS users(
     id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE,
     name TEXT,
     age INTERGER
)
""")
conn.commit()


#### Supprimer la table 

In [5]:
'''
cursor = conn.cursor()
cursor.execute("""
DROP TABLE users
""")
conn.commit()
'''

'\ncursor = conn.cursor()\ncursor.execute("""\nDROP TABLE users\n""")\nconn.commit()\n'

#### Insérer des data 

Il existe plusieurs manière d'insérer des données, la plus simple étant celle-ci :

In [6]:
cursor.execute("""
INSERT INTO users(name, age) VALUES(?, ?)""", ("olivier", 30))

#### Vous pouvez passer par un dictionnaire :

In [7]:
data = {"name" : "olivier", "age" : 30}
cursor.execute("""
INSERT INTO users(name, age) VALUES(:name, :age)""", data)

#### Vous pouvez récupérer l'id de la ligne que vous venez d'insérer de cette manière :

In [8]:
id = cursor.lastrowid
print('dernier id: %d' % id)

dernier id: 2


#### Il est également possible de faire plusieurs insert en une seule fois avec la fonction executemany :


In [9]:
users = []
users.append(("olivier", 30))
users.append(("jean-louis", 90))
cursor.executemany("""
INSERT INTO users(name, age) VALUES(?, ?)""", users)

#### Récupérer des données

Vous pouvez récupérer la première ligne correspondant à votre recherche à l'aide de **la fonction fetchone.**

In [10]:
cursor.execute("""SELECT name, age FROM users""")
user1 = cursor.fetchone()
#print le TUPLE
print(user1)

('olivier', 30)


#### Récupérer plusieur data 
Vous pouvez récupérer plusieurs données de la même recherche en utilisant **la fonction fetchall().**

In [11]:
cursor.execute("""SELECT id, name, age FROM users""")
rows = cursor.fetchall()
for row in rows:
    print('{0} : {1} - {2}'.format(row[0], row[1], row[2]))

1 : olivier - 30
2 : olivier - 30
3 : olivier - 30
4 : jean-louis - 90


In [12]:
#L'objet curseur fonctionne comme un itérateur, invoquant la méthode fetchall() automatiquement 
cursor.execute("""SELECT id, name, age FROM users""")
for row in cursor:
    print('{0} : {1}, {2}'.format(row[0], row[1], row[2]))


1 : olivier, 30
2 : olivier, 30
3 : olivier, 30
4 : jean-louis, 90


In [13]:
#Pour la recherche spécifique,on utilise la même logique vu précédemment :
id = 2
cursor.execute("""SELECT id, name FROM users WHERE id=?""", (id,))
response = cursor.fetchone()

#### Gestion des erreurs 


In [14]:
try:
    conn = sqlite3.connect('db/users.db')
    cursor = conn.cursor()
    cursor.execute("""
CREATE TABLE users(
    id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE,
    name TEXT,
    age INTERGER
)
""")
    conn.commit()
except sqlite3.OperationalError:
    print('Erreur la table existe déjà')
except Exception as e:
    print("Erreur")
    conn.rollback()
    # raise e
finally:
    conn.close()

Les erreurs que vous pouvez intercepter :
```
Error
DatabaseError
DataError
IntegrityError
InternalError
NotSupportedError
OperationalError
ProgrammingError
InterfaceError
Warning
```

### Les fonctions ca facilite la vie quand même

In [15]:
DEFAULT_PATH = os.path.join(os.path.dirname("./db/ma_base_exemple.db"), 'database.sqlite3')

def db_connect(db_path=DEFAULT_PATH):  
    con = sqlite3.connect(db_path)
    return con


## Un cas d'application simple : La major company 

**Voir le schéma sur le diapo du cours**


On a donc 11 tables dans notre template de base de données :  
- La table des employés stocke les données relatives aux employés, telles que l'ID d'employé, le nom de famille, le prénom, etc. Elle comporte également un champ nommé ReportsTo pour spécifier qui fait rapport à qui.
- La table des clients stocke les données des clients.
- La tables invoices & facture_items: ces deux tables stockent les données de facturation, la table de factures stocke les données d'en-tête de facture et la table invoice_items stocke les données d'élément de ligne de facture
- La table artistes table stocke les données sur les artistes. C'est un tableau simple qui ne contient que l'identifiant et le nom de l'artiste.
- La table des albums stocke des données sur une liste de pistes. Chaque album appartient à un artiste. Cependant, un artiste peut avoir plusieurs albums.
- La table media_types stocke les types de média tels que l'audio MPEG et le fichier audio AAC.
 genres table stocke des types de musique tels que le rock, le jazz, le métal, etc.
- La table des pistes stocke les données des chansons. Chaque piste appartient à un album.
 playlists & playlist_track tables: playlists table stocke des données sur les playlists. Chaque playlist contient une liste de pistes. Chaque piste peut appartenir à plusieurs playlists. La relation entre la table des listes de lecture et la table des pistes est multiple. La table playlist_track est utilisée pour refléter cette relation.



#### Explorer votre base de données 

Lancer la commande suivante pour accéder à l'interface sqlite3 à la base : 
```
sqlite3 db/chinook.db
```

Regarder le contenu de la base : 
```
.tables
```

L'instruction suivante renvoie la table qui se termine par la chaîne 'es' : 
```
.tables '%es' 
```

Regarder la structure de la table des albums

```
.schema albums
```

Pour avoir la structure générale du schéma : 

```
.fullschema
```

Pour avoir les clefs primaire : 
```
.indexes
```

Mettre sa base de donnée oklm dans un fichier txt : 
```
.output albums.txt
SELECT title FROM albums;
```

#### Les commandes principales 

** SELECT **

On utilisera principalement l'instruction *SELECT* pour interroger les données d'une ou de plusieurs tables. La syntaxe de l'instruction SELECT est la suivante:
```
SELECT DISTINCT column_list
FROM table_list
  JOIN table ON join_condition
WHERE row_filter
ORDER BY column
LIMIT count OFFSET offset
GROUP BY column
HAVING group_filter;
```

Mais on peut faire aussi un SELECT plus classique : 
```
SELECT
 trackid,
 name,
 composer,
 unitprice
FROM
 tracks;
```

Et bien sur le quand classico : 
```
SELECT
 *
FROM
 tracks;
```
avec le ** * ** qui signifie ALL


** ORDER BY **

La clause ORDER BY vient après la clause FROM. La clause ORDER BY vous permet de trier le jeu de résultats en fonction d'une ou de plusieurs colonnes d'ordres différents: croissant et décroissant.

```
SELECT
 column_list
FROM
 table
ORDER BY
 column_1 ASC,
 column_2 DESC;
```

Supposons qu'on veut trier le jeu de résultats en fonction de la colonne *AlbumId* par ordre croissant, il faut donc  utiliser l'instruction suivante:
```
SELECT
 name,
 milliseconds, 
 albumid
FROM
 tracks
ORDER BY
 albumid ASC;
```


** WHERE **

La clause WHERE est une clause facultative de l'instruction SELECT. Il apparaît après la clause FROM sous la forme suivante:

```
SELECT
 column_list
FROM
 table
WHERE
 search_condition;
```

On peut utiliser des condistion assez folklorique du type 
```
WHERE column_1 = 100;
 
WHERE column_2 IN (1,2,3);
 
WHERE column_3 LIKE 'An%';
 
WHERE column_4 BETWEEN 10 AND 20;
```


** BETWEEN **

L'opérateur BETWEEN est un opérateur logique qui teste si une valeur est dans la plage de valeurs. Si la valeur est dans la plage spécifiée, l'opérateur BETWEEN renvoie la valeur true. 
```
test_expression BETWEEN low_expression AND high_expression
```
L'opérateur BETWEEN peut être utilisé dans la clause WHERE des instructions SELECT, DELETE, UPDATE et REPLACE.

```
SELECT
    InvoiceId,
    BillingAddress,
    Total
FROM
    invoices
WHERE
    Total BETWEEN 14.91 and 18.86    
ORDER BY
    Total; 
```
ou encore avec le *NOT BETWEEN* : 
```
SELECT
    InvoiceId,
    BillingAddress,
    Total
FROM
    invoices
WHERE
    Total NOT BETWEEN 1 and 20
ORDER BY
    Total; 
```


** INNER JOIN **

Pour interroger des données de plusieurs tables, utiliser la clause INNER JOIN est assez intuitif. La clause INNER JOIN combine les colonnes des tables corrélées.

Supposons deux tables: A et B.

A a les colonnes a1, a2 et f. B a les colonnes b1, b2 et f. La table A est liée à la table B à l'aide d'une colonne de clé étrangère nommée f. Soit : 

```
SELECT a1, a2, b1, b2
FROM A
INNER JOIN B on B.f = A.f;
```

**Petit point sur les jointure, un bon schéma vaut toujours mieux qu'un long discours**
<img src="https://i2.wp.com/www.codeproject.com/KB/database/Visual_SQL_Joins/Visual_SQL_JOINS_V2.png">

** HAVING **

La clause SQLite HAVING est une clause facultative de l'instruction SELECT. La clause HAVING spécifie une condition de recherche pour un groupe.
```
SELECT
 column_1,
 aggregate_function (column_2)
FROM
 table
GROUP BY
 column_1
HAVING
 search_condition;
```

appliqué à notre exemple de base : 
```
SELECT
 albumid,
 COUNT(trackid)
FROM
 tracks
GROUP BY
 albumid;
```

encore plus loin : 
```
SELECT
 albumid,
 COUNT(trackid)
FROM
 tracks
GROUP BY
 albumid
HAVING count(albumid) BETWEEN 18 AND 20
ORDER BY albumid;
```


** GROUP BY **

Pareil, la clause GROUP BY est une clause facultative de l'instruction SELECT. Elle GROUP BY regroupe un groupe de lignes sélectionné en lignes de synthèse contenant les valeurs d'une ou de plusieurs colonnes.

```
SELECT
 albumid,
 COUNT(trackid)
FROM
 tracks
GROUP BY
 albumid;
```

toujours plus : 
```
SELECT
 tracks.albumid,
 title,
 COUNT(trackid)
FROM
 tracks
INNER JOIN albums ON albums.albumid = tracks.albumid
GROUP BY
 tracks.albumid;
```

encore des petits tricks : 

```
SELECT
 tracks.albumid,
 title,
 min(milliseconds),
 max(milliseconds),
 round(avg(milliseconds),2)
FROM
 tracks
INNER JOIN albums ON albums.albumid = tracks.albumid
GROUP BY
 tracks.albumid;
```


ou encore très simple : 
```
SELECT
 mediatypeid,
 genreid,
 count(trackid)
FROM
 tracks
GROUP BY
 mediatypeid,
 genreid;
```


### Questions 'mini TP'  

** Le mini TP sera à rendre par mail en format MARKDOWN vous avez jusqu'au 06/03/2019 18h pour me rendre ce mini TP ainsi que votre mini projet**

- Donner la commande qui permet de lister les tables présente dans le fichier chinook.db
- Donner la commande qui permet de lister les tables se terminant par 's'
- Donner la commande qui permet de visualiser la construction de la table 'albums'
- Donner la commande qui permet de visualiser la construction de toutes les tables 
- Donner la commande qui permet de selectionner toutes les occurrences de la table TRACKS 
- Donner l'instruction qui permet de selectionner les 10 premiers 'albumID' de la table albums ordonné par 'artistID' 
- Donner la commande qui permet de selectionner les 10 premiers nom de la table tracks ordonné par 'albumid'
- Donner la commande qui permet de selectionner les 10 premiers nom de la table tracks ou 'albumid' = 1 
- Donner la commande qui permet de selectionner les 10 premiers nom de la table tracks ou 'albumid' = 1 et la durée du son est > 252980 ms
- Donner la commande qui permet de selectionner les 10 premiers nom de la table tracks ou le 'type media' est 1 et/ou 2 ordonné par nom de tracks
- Donner la commande qui permet de selectionner les artistes, les ID de ces artistes et les ID des albums (de la table artiste) tel que les ID des artistes soit les mêmes que les ID de leurs albums. Tout cela ordonné par ID album.  

#### Petit plus : gestion du CSV 

Exporter votre base de données vers un fichier CSV :

- Activez l'en-tête du jeu de résultats à l'aide de la commande .header on.
- Définissez le mode de sortie sur CSV pour demander à l'outil sqlite3 d'émettre le résultat en mode CSV.
- Envoyez la sortie dans un fichier CSV.
- Lancez la requête pour sélectionner les données de la table vers laquelle vous souhaitez exporter.

```
>sqlite3 c:/sqlite/chinook.db
sqlite> .headers on
sqlite> .mode csv
sqlite> .output data.csv
sqlite> SELECT customerid,
   ...>        firstname,
   ...>        lastname,
   ...>        company
   ...>   FROM customers;
sqlite> .quit
```
